In [ ]:
import numpy as np
from scipy.integrate import odeint

class CarParking:
    def __init__(self, delta=0.1):
        """
        Initialize the environment parameters.
        :param delta: time step (seconds)
        """
        self.delta = delta
        self.state_range = [-5,5]
        self.control_inputs = np.array([-5, -1, -0.1, -0.01, -0.001, -0.0001, 0,
                                        0.0001, 0.001, 0.01, 0.1, 1, 5])
        self.X = np.linspace(-5, 5, 21)
        self.V = np.linspace(-5, 5, 21)


    def model(self, s, t, u):
        """
        ODE for the car
        s: state[x,v]
        u: acceleration
        t: time
        """
        dsdt = [s[1], u]
        return dsdt

    def step(self, state, action):
        """
        Apply one control action for one time step(delta=0.1)
        state: current [x,v]
        action: acceleration (u)
        returns next_state
        """
        t = np.linspace(0, 0.1, 2)
        next_state = odeint(self.model, state, t, args=(action,))[-1]
        next_state = np.clip(next_state, self.state_range[0], self.state_range[1])
        return next_state


    def discretize_state(self, state):
        """
        Map continuous state to discrete indices for Q-table lookup
        state: [x,u]
        returns (x_idx, y_idx)
        """
        x, v = state
        #finding the nearest gridpoints
        x_idx = np.argmin(np.abs(self.X - x))
        v_idx = np.argmin(np.abs(self.V - v))
        return x_idx, v_idx


    def get_trajectory(self, s0, q_table, max_steps=100):
        """
        Generate a greedy approach given a Q-table
        s0: initial state[x0, v0]
        q_table: shape
        max_steps: maximum steps
        returns: list of (state, action_idx, reward)
        """
        trajectory = [s0]
        s = s0
        for _ in range(max_steps):
            x_idx, v_idx = self.discretize_state(s)
            a_idx = np.argmax(q_table[x_idx, v_idx])
            u = self.control_inputs[a_idx]
            s = self.step(s,u)
            trajectory.append(s)
            if np.linalg.norm(s) < 0.01:
                break
        return trajectory

In [ ]:
def calculate_reward(state):
    """
    Calculte reward
    state: [x,v]
    """
    x, v = state
    if np.linalg.norm(state) < 0.01:
        return 100
    return -(x**2 + v**2)

In [ ]:
import numpy as np
import random
from tqdm import tqdm

class QLearningAgent:
    def __init__(self, env, learning_rate=0.01, discount_factor=0.9, epsilon_greedy=0.9,
                 epsilon_min=0.1, epsilon_decay=0.995):
        self.env = env
        self.lr = learning_rate
        self.gamma = discount_factor
        self.epsilon = epsilon_greedy
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay
        ## Define the q_table
        self.q_table = np.zeros((len(env.X), len(env.V), len(env.control_inputs)))

    def choose_action(self, state):
        x_idx, v_idx = self.env.discretize_state(state)
        if random.uniform(0, 1) < self.epsilon:
            return random.randint(0, len(self.env.control_inputs) - 1)
        else:
            q_vals = self.q_table[x_idx, v_idx]
            perm_actions = np.random.permutation(len(self.env.control_inputs))
            q_vals_perm = [q_vals[a] for a in perm_actions]
            return perm_actions[np.argmax(q_vals_perm)]

    def _learn(self, transition):
        """
        Updates Q-table using TD learning rule:
        Q(s,a) <- Q(s,a) + α [r + γ max_a' Q(s',a') - Q(s,a)]
        """
        s, a, r, next_s, done = transition
        # Discretize current and next states for indexing
        x_idx, v_idx = self.env.discretize_state(s)
        next_x_idx, next_v_idx = self.env.discretize_state(next_s)

        #Current Q-value
        q_val = self.q_table[x_idx, v_idx, a]
        if done:
            #Terminal state
            q_target = r
        else:
            #Non-terminal state
            q_target = r + self.gamma * np.max(self.q_table[next_x_idx, next_v_idx])
        #Update rule
        self.q_table[x_idx, v_idx, a] += self.lr * (q_target - q_val)

        #Decay exploration rate
        self._adjust_epsilon()

    def _adjust_epsilon(self):
        """
        Gradually reduce ε to a minimum for balcncing exploration and exploitation
        """
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def train(self, episodes=10000, max_steps=500):
        """
        Run Q-leaning for a given number of episodes
        """
        for _ in tqdm(range(episodes)):
            state = np.random.uniform(-5, 5, size=2)  # Random start
            for _ in range(max_steps):
                a_idx = self.choose_action(state)
                u = self.env.control_inputs[a_idx]
                next_state = self.env.step(state, u)
                reward = calculate_reward(next_state)
                done = np.linalg.norm(next_state) < 0.01

                transition = (state, a_idx, reward, next_state, done)
                self._learn(transition)

                state = next_state
                if done:
                    break

    def print_q_table(self):
        """
        Prints the entire Q-table
        """
        print(self.q_table)

In [ ]:
env = CarParking()
agent = QLearningAgent(env)
agent.train(episodes=10000, max_steps=500)

100%|████████████████████████████████████████████████████████████████████████████| 10000/10000 [04:56<00:00, 33.73it/s]


In [ ]:
def evaluate_policy(env, q_table, test_states=None):
    """
    Evaluate a learned Q-policy by executing its greedy policy on specified inintial state
    env: CarParking environment
    q_table: Q- table
    test_states: list of inital [x,v] states for testing
    returns: (success_rate, average steps)
            success_rate: fraction of test episodes that reached the goal
            average_steps: average number of steps taken per episode
    """
    if test_states is None:
        test_states = [[5,5]]

    successes = 0
    total_steps = 0

    for s0 in test_states:
        s = s0
        for _ in range(20000):  # max steps per episode
            x_idx, v_idx = env.discretize_state(s)
            state_key = (x_idx, v_idx)
            if isinstance(q_table, np.ndarray):
                a_idx = np.argmax(q_table[x_idx, v_idx])
            else:
                a_idx = np.argmax(q_table[state_key])

            u = env.control_inputs[a_idx]
            s = env.step(s, u)
            total_steps += 1
            if np.linalg.norm(s) < 0.01:
                successes += 1
                break

    #Overall metrics
    success_rate = successes / len(test_states)
    avg_steps = total_steps / len(test_states)
    return success_rate, avg_steps


In [ ]:
success_rate, avg_steps = evaluate_policy(env, agent.q_table)
print(f"✅ Success Rate: {success_rate*100:.2f}%")
print(f"📏 Average Steps to Park: {avg_steps:.2f}")

✅ Success Rate: 0.00%
📏 Average Steps to Park: 20000.00


In [ ]:
import numpy as np
from tqdm import tqdm
import itertools
import pandas as pd

# Hyperparameters
grid_sizes = [21, 51, 101]
learning_rates = [0.1, 0.01]
discount_factors = [0.9, 0.99]
training_configs = [(3000, 300), (5000, 500), (10000, 500)]

# Results
results = []

# Training and Evaluation
for size in grid_sizes:
    for lr in learning_rates:
        for gamma in discount_factors:
            for episodes, max_steps in training_configs:
                print(f"\nGrid={size}, LR={lr}, Gamma={gamma}, Episodes={episodes}, MaxSteps={max_steps}")

                # Create environment and set grid
                env = CarParking(delta=0.1)
                env.X = np.linspace(-5, 5, size)
                env.V = np.linspace(-5, 5, size)

                # Initialize agent
                agent = QLearningAgent(env, learning_rate=lr, discount_factor=gamma,
                                       epsilon_greedy=1.0, epsilon_min=0.1, epsilon_decay=0.995)

                # Train
                agent.train(episodes=episodes, max_steps=max_steps)

                # Evaluation
                values = [-5, -4, -3.4, -3, -2.5, 0.0, 1, 2, 2.5, 3.5, 4.5, 5]
                test_states = [[x, v] for x, v in itertools.product(values, values)]

                successes = 0
                total_steps = 0

                for s0 in test_states:
                    s = s0
                    for step_count in range(20000):
                        x_idx, v_idx = env.discretize_state(s)
                        a = np.argmax(agent.q_table[x_idx, v_idx])
                        u = env.control_inputs[a]
                        s = env.step(s, u)
                        if np.linalg.norm(s) < 0.01:
                            successes += 1
                            total_steps += step_count
                            break

                success_rate = successes / len(test_states)
                avg_steps = total_steps / successes if successes > 0 else None

                results.append({
                    "grid_size": size,
                    "learning_rate": lr,
                    "discount_factor": gamma,
                    "episodes": episodes,
                    "max_steps": max_steps,
                    "success_rate": success_rate,
                    "avg_steps": avg_steps
                })

#Results
df = pd.DataFrame(results)
print(df)



Grid=21, LR=0.1, Gamma=0.9, Episodes=3000, MaxSteps=300


100%|██████████████████████████████████████████████████████████████████████████████| 3000/3000 [00:55<00:00, 54.39it/s]



Grid=21, LR=0.1, Gamma=0.9, Episodes=5000, MaxSteps=500


100%|██████████████████████████████████████████████████████████████████████████████| 5000/5000 [02:29<00:00, 33.35it/s]



Grid=21, LR=0.1, Gamma=0.9, Episodes=10000, MaxSteps=500


...


Grid=101, LR=0.01, Gamma=0.99, Episodes=5000, MaxSteps=500


100%|██████████████████████████████████████████████████████████████████████████████| 5000/5000 [03:07<00:00, 26.60it/s]



Grid=101, LR=0.01, Gamma=0.99, Episodes=10000, MaxSteps=500


100%|████████████████████████████████████████████████████████████████████████████| 10000/10000 [05:28<00:00, 30.42it/s]


    grid_size  learning_rate  discount_factor  episodes  max_steps  \
0          21           0.10             0.90      3000        300   
1          21           0.10             0.90      5000        500   
2          21           0.10             0.90     10000        500   
3          21           0.10             0.99      3000        300   
4          21           0.10             0.99      5000        500   
5          21           0.10             0.99     10000        500   
6          21           0.01             0.90      3000        300   
7          21           0.01             0.90      5000        500   
8          21           0.01             0.90     10000        500   
9          21           0.01             0.99      3000        300   
10         21           0.01             0.99      5000        500   
11         21           0.01             0.99     10000        500   
12         51           0.10             0.90      3000        300   
13         51       